In [2]:
!pip3 install dlib 

    100% |████████████████████████████████| 3.3MB 332kB/s ta 0:00:011
  Running setup.py bdist_wheel for dlib ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /home/leaozinho/.cache/pip/wheels/ce/f9/bc/1c51cd0b40a2b5dfd46ab79a73832b41e7c3aa918a508154f0
Successfully built dlib


In [14]:
import cv2
import os
import glob
import _pickle as cPickle
import dlib
import numpy as np

In [15]:
np.__version__
!ls

delirium    Imagens-e-recursos.zip  relogios_teste	  Untitled.ipynb
dlib-19.10  __MACOSX		    relogios_teste2	  yalefaces
fotos	    recursos		    relogios_treinamento


In [ ]:
detectorface  = dlib.get_frontal_face_detector()
detectorpontos = dlib.shape_predictor("./recursos/shape_predictor_68_face_landmarks.dat")
reconhecimentofacial = dlib.face_recognition_model_v1("./recursos/dlib_face_recognition_resnet_model_v1.dat")
indices = np.load("./recursos/indices_rn.pickle")
descritoresfaciais = np.load("./recursos/descritores_rn.npy")


for arquivo in glob.glob(os.path.join("./fotos", "*.jpg")):
  imagem = cv2.imread(arquivo)
  facesdetectadas = detectorface(imagem, 2)
  for face in facesdetectadas:
    e, t, d, b = (int(face.left()), int(face.top()), int(face.right()), int(face.bottom()))
    pontosfaciais = detectorpontos(imagem, face)
    
    descritorfacial = reconhecimentofacial.compute_face_descriptor(imagem, pontosfaciais)
    
    ldf = [fd for fd in descritorfacial]
    
    npa = np.asarray(ldf, dtype=np.float64)
    npa = npa[np.axis, :]
    distancias = np.linalg.norm(npa - descritoresfaciais, axis=1)
    print("distancias: {} ".format(distancias))
    
    
    cv2.rectangle(imagem, (e, t), (d, b), (255, 0, 255), 2)
    plt.imshow(imagem)
    
    

In [ ]:
import os
import glob
import _pickle as cPickle
import dlib
import cv2
import numpy as np

detectorFace = dlib.get_frontal_face_detector()
detectorPontos = dlib.shape_predictor("recursos/shape_predictor_68_face_landmarks.dat")
reconhecimentoFacial = dlib.face_recognition_model_v1("recursos/dlib_face_recognition_resnet_model_v1.dat")
indices = np.load("recursos/indices_rn.pickle")
descritoresFaciais = np.load("recursos/descritores_rn.npy")
limiar = 0.5

for arquivo in glob.glob(os.path.join("fotos", "*.jpg")):
    imagem = cv2.imread(arquivo)
    facesDetectadas = detectorFace(imagem, 2)
    for face in facesDetectadas:
        e, t, d, b = (int(face.left()), int(face.top()), int(face.right()), int(face.bottom()))
        pontosFaciais = detectorPontos(imagem, face)
        descritorFacial = reconhecimentoFacial.compute_face_descriptor(imagem, pontosFaciais)
        listaDescritorFacial = [fd for fd in descritorFacial]
        npArrayDescritorFacial = np.asarray(listaDescritorFacial, dtype=np.float64)
        npArrayDescritorFacial = npArrayDescritorFacial[np.newaxis, :]

        distancias = np.linalg.norm(npArrayDescritorFacial - descritoresFaciais, axis=1)
        print("Distâncias: {}".format(distancias))
        minimo = np.argmin(distancias)
        print(minimo)
        distanciaMinima = distancias[minimo]
        print(distanciaMinima)

        if distanciaMinima <= limiar:
            nome = os.path.split(indices[minimo])[1].split(".")[0]
        else:
            nome = ' '

        cv2.rectangle(imagem, (e, t), (d, b), (0, 255, 255), 2)
        texto = "{} {:.4f}".format(nome, distanciaMinima)
        cv2.putText(imagem, texto, (d, t), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (0, 255, 255))

    cv2.imshow("Detector hog", imagem)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Distâncias: [0.40523834 0.41441994 0.40148712 0.85975471 0.8619478  0.88271185
 0.84777851 0.86983831]
2
0.4014871205302875
Distâncias: [0.74006643 0.74497679 0.77336236 0.41498253 0.41599224 0.4259358
 0.41661208 0.42673303]
3
0.41498253124881856


In [ ]:
import os
import glob
import _pickle as cPickle
import dlib
import cv2
import numpy as np
from PIL import Image

detectorFace = dlib.get_frontal_face_detector()
detectorPontos = dlib.shape_predictor("recursos/shape_predictor_68_face_landmarks.dat")
reconhecimentoFacial = dlib.face_recognition_model_v1("recursos/dlib_face_recognition_resnet_model_v1.dat")
indices = np.load("recursos/indices_yales.pickle")
descritoresFaciais = np.load("recursos/descritores_yale.npy")
#print(descritoresFaciais)
limiar = 0.5
totalFaces = 0
totalAcertos = 0

for arquivo in glob.glob(os.path.join("yalefaces/teste", "*.gif")):
    imagemFace = Image.open(arquivo).convert('RGB')
    imagem = np.array(imagemFace, 'uint8')
    idatual = int(os.path.split(arquivo)[1].split(".")[0].replace("subject", ""))
    totalFaces += 1

    facesDetectadas = detectorFace(imagem, 2)
    for face in facesDetectadas:
        e, t, d, b = (int(face.left()), int(face.top()), int(face.right()), int(face.bottom()))
        
        pontosFaciais = detectorPontos(imagem, face)
        descritorFacial = reconhecimentoFacial.compute_face_descriptor(imagem, pontosFaciais)
        listaDescritorFacial = [fd for fd in descritorFacial]
        npArrayDescritorFacial = np.asarray(listaDescritorFacial, dtype=np.float64)
        npArrayDescritorFacial = npArrayDescritorFacial[np.newaxis, :]
#        print(npArrayDescritorFacial)
        #print(descritorFacial)
        distancias = np.linalg.norm(npArrayDescritorFacial - descritorFacial, axis=1)
        #print("Distâncias: {}".format(distancias))
        minimo = np.argmin(distancias)
        #print(minimo)
        distanciaMinima = distancias[minimo]
        #print(distanciaMinima)

        if distanciaMinima <= limiar:
            nome = os.path.split(indices[minimo])[1].split(".")[0]
            idprevisto = int(os.path.split(indices[minimo])[1].split(".")[0].replace("subject", ""))
            if idprevisto == idatual:
                totalAcertos += 1
        else:
            nome = 'Desconhecido'

        print("idatual: {} idprevisto: {}".format(idatual, idprevisto))
        cv2.rectangle(imagem, (e, t), (d, b), (0, 0, 255), 2)
        texto = "{} {:.4f}".format(nome, distanciaMinima)
        cv2.putText(imagem, texto, (d, t), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (0, 0, 255))

    cv2.imshow("Detector hog", imagem)
    cv2.waitKey(0)

percentualAcerto = (totalAcertos / totalFaces) * 100
print("Percentual de acerto: {}".format(percentualAcerto))
cv2.destroyAllWindows()

None
[[-1.15775600e-01  1.17102943e-01 -9.49815847e-04 -3.88331339e-02
  -1.14998057e-01 -6.49198443e-02  1.04906969e-02 -6.76791295e-02
   1.96887866e-01  1.38110556e-02  2.50860512e-01 -7.53434300e-02
  -3.02676052e-01 -1.91650186e-02 -8.66417587e-02  1.11534111e-01
  -1.49563640e-01 -4.46849130e-02 -1.42389730e-01 -1.37142345e-01
   4.07816730e-02  8.03309903e-02  4.56940271e-02 -1.08663701e-02
  -1.74881041e-01 -3.15513581e-01 -7.61291757e-02 -1.20651148e-01
   7.02237189e-02 -1.46277770e-01 -1.80993266e-02  8.80828723e-02
  -1.62865967e-01 -1.04021430e-01  1.23766605e-02  7.59848505e-02
  -5.22349216e-02 -5.00274524e-02  2.00931251e-01  7.90067762e-02
  -1.52099922e-01  4.50811610e-02  1.04943678e-01  3.21350604e-01
   1.44640490e-01  1.74997784e-02  2.50652060e-02 -8.65318999e-02
   8.85159224e-02 -2.44557455e-01  1.48338377e-01  1.59796894e-01
   9.11264345e-02  7.54504800e-02  1.03001855e-01 -2.08735988e-01
  -1.40110385e-02  1.20441549e-01 -2.74558336e-01  1.10195436e-01
   1.